<a href="https://colab.research.google.com/github/smarttransit-ai/smarttransit-ai.github.io/blob/master/files/notebooks/Macro_Prediction_Models/Generating_Samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas
!apt install libspatialindex-dev
!pip install rtree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [2]:
import pandas as pd
import math
import tqdm

import os, time
import geopandas as gpd
import shutil
from rtree import index

from IPython.display import display, HTML
import pandas as pd
import numpy as np
from  more_itertools import unique_everseen

from shapely.geometry import *

import datetime
import geopy.distance



In [3]:
#obtain the repository
!git clone https://github.com/smarttransit-ai/smarttransit-ai.github.io.git

Cloning into 'smarttransit-ai.github.io'...
remote: Enumerating objects: 1180, done.
remote: Total 1180 (delta 0), reused 0 (delta 0), pack-reused 1180
Receiving objects: 100% (1180/1180), 399.10 MiB | 36.10 MiB/s, done.
Resolving deltas: 100% (420/420), done.
Checking out files: 100% (625/625), done.


In [4]:
Vehicle_Type = 'Electric_Vehicles'
Vehicle_ID = [751]
datetimeFormat = '%Y-%m-%d %H:%M:%S:%f'
%cd

/root


In [5]:
def addingSegmentNumbers(df):
    OSM_ID_list = list(df.OSM_ID)
    print(len(OSM_ID_list))

    OSM_ID = []
    for i in tqdm.tqdm(OSM_ID_list):
        OSM_ID.append(str(i))
    print(len(OSM_ID))

    OSM_Feature = []
    Segment = []

    S = 1
    i = 0
    null = 1
    while i < len(OSM_ID):
        OSM_ID[i]=str(OSM_ID[i])
        if i >len(OSM_ID)-1:
            break
        if len(OSM_ID[i]) > 1:
            OSM_Feature.append(OSM_ID[i])

            for j in range(i,len(OSM_ID)):

                if j == len(OSM_ID)-1:
                    i = j
                    Segment.append(S)
                    null = S
                    break
                elif OSM_ID[i] != OSM_ID[j] and len(OSM_ID[j]) != 1:
                    i = j-1
                    S = S + 2
                    break
                else:
                    Segment.append(S)
        else:
            Segment.append(null)
        i += 1

    print(f'len of OSM_feature = {len(OSM_Feature)}')
    print(len(OSM_ID))
    print(len(Segment))
    return Segment

    

In [7]:
%cd
%cd /content/smarttransit-ai.github.io/files/dataset/Electric_Vehicles/

for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')

    csv = f"{Vehicle_Name}_Predicted_Data_for_Map_Matching.csv"
    df = pd.read_csv(csv,low_memory=False)
    print(len(df))

    Segment = addingSegmentNumbers(df)
    df['Segment_Number'] = Segment

    df.to_csv(f'{Vehicle_Name}_after_Identifying_Segments.csv', index=False)


/root
/content/smarttransit-ai.github.io/files/dataset/Electric_Vehicles
Processing BYD_751


100%|██████████| 81553/81553 [00:00<00:00, 1716202.69it/s]

81553
81553
81553
len of OSM_feature = 2176
81553
81553


1. Calculating Distance Travelled, Charge Consumed, Time Taken for each sample (Continuous Travel on the Same OSM Segment) 
2. One-hot encoding Road Types for each sample 

In [ ]:
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')

    df = pd.read_csv(f'{Vehicle_Name}_after_Identifying_Segments.csv', low_memory=False)
    print(len(df))

    print(df.columns)
    Latitude = list(df.Latitude)
    Longitude = list(df.Longitude)

    F_Class = []
    OSM_Feature = []

    Initial_Latitude = []
    Initial_Longitude = []
    Final_Latitude = []
    Final_Longitude =[]


    TimeStamp_Begin = []
    TimeStamp_End = []
    TimeStart = []
    TimeFinish = []
    Charge_at_Begining = []
    Charge_at_Final = []

    Distance = []
    TimeNeeded_in_Seconds = []
    Oneway = []
    Max_Speed = []
    Tunnel = []

    Charge_Consumed = []
    Type_of_Road = []
    datetimeFormat = '%Y-%m-%d %H:%M:%S:%f'
    NewSegment = []
    i = 0

    while i < (len(df.Segment_Number)):
        LatLong_individual = []
        if not math.isnan(Latitude[i]):
            time_start = df.Time_US_Central[i]
            TimeStart.append(time_start)
            OSM_Feature.append(df.OSM_ID[i])
            F_Class.append(df.F_Class[i])
            TimeStamp_Begin.append(df.Unix_Timestamp_ms[i])
            initial_charge = df.SOC[i]
            Charge_at_Begining.append(initial_charge)

            D = 0
            NewSegment.append(df.Segment_Number[i])
            coords_1 = (Latitude[i], Longitude[i])
            Initial_Latitude.append(Latitude[i])
            Initial_Longitude.append(Longitude[i])
            Point_Shape = Point(Longitude[i], Latitude[i])
            LatLong_individual.append(Point_Shape)

            for j in range(i,len(df.Segment_Number)):
                if j == len(df.OSM_ID)-1:
                    End_location = j
                    while End_location >= i:
                        if not math.isnan(Latitude[End_location]):
                            coords_2 = (Latitude[End_location], Longitude[End_location])
                            Final_Latitude.append(Latitude[End_location])
                            Final_Longitude.append(Longitude[End_location])
                            time_end = df.Time_US_Central[End_location]
                            TimeFinish.append(time_end)
                            TimeStamp_End.append(df.Unix_Timestamp_ms[End_location])
                            final_charge = df.SOC[End_location]
                            Charge_at_Final.append(final_charge)
                            break
                        else:
                            End_location = End_location-1
                    i = j+1
                    break
                if df.Segment_Number[i] != df.Segment_Number[j]:
                    k = j-1
                    while k >= i:
                        if not math.isnan(Latitude[k]):
                            coords_2 = (Latitude[k], Longitude[k])
                            Final_Latitude.append(Latitude[k])
                            Final_Longitude.append(Longitude[k])
                            time_end = df.Time_US_Central[k]
                            TimeFinish.append(time_end)
                            TimeStamp_End.append(df.Unix_Timestamp_ms[k])
                            final_charge = df.SOC[k]
                            Charge_at_Final.append(final_charge)
                            break
                        else:
                            k = k-1
                    i = j-1
                    break


            TD = datetime.datetime.strptime(time_end, datetimeFormat) - datetime.datetime.strptime(time_start, datetimeFormat)
            C = initial_charge - final_charge

            Charge_Consumed.append(C)
            TimeNeeded_in_Seconds.append(TD.seconds)
            D = geopy.distance.distance(coords_1, coords_2).m
            Distance.append(D)


        if len(df.Segment_Number) == len(Distance):
            break

        i +=1


    print('***************************')
    print(f'Total Segment = {len(df.Segment_Number)}')
    print(f'Length of Distance Array = {len(Distance)}')
    # print(TimeNeeded_in_Seconds)
    print(f'Length of TimeNeeded_in_Seconds = {len(TimeNeeded_in_Seconds)}')
    print(f'Length of Charge_Consumed = {len(Charge_Consumed)}')
    print(f'Length of different Segments = {len(NewSegment)}')
    print(len(F_Class))


    TrainingSample = pd.DataFrame({"Segment": NewSegment,
                       "Distance":Distance,
                        "OSM_Feature":OSM_Feature,
                         "FClass" :F_Class,
                        "Initial_recorded_Latitude":Initial_Latitude,
                        "Initial_recorded_Longitude":Initial_Longitude,
                        "Final_recorded_Latitude":Final_Latitude,
                        "Final_recorded_Longitude":Final_Longitude,
                       "TimeStamp_Begin":TimeStamp_Begin,
                       "TimeStamp_End":TimeStamp_End,
                        "TimeStart":TimeStart,
                        "TimeFinish":TimeFinish,
                        "Charge_at_Begining":Charge_at_Begining,
                        "Charge_at_Final":Charge_at_Final,
                       "TimeNeeded_in_Seconds":TimeNeeded_in_Seconds,
                        "Charge_Consumed":Charge_Consumed})

    one_hot_FClass = pd.get_dummies(TrainingSample.FClass)

    TrainingSample = TrainingSample.join(one_hot_FClass)

    print(TrainingSample.columns)
    

    TrainingSample.to_csv(f'{Vehicle_Name}_with_Segment_Number_without_Turns.csv',index=False)


Processing BYD_751
81553
Index(['Location_Index', 'Latitude', 'Longitude', 'Unix_Timestamp_ms',
       'Time_US_Central', 'SOC', 'Candidate', 'OSM_ID', 'Street_Name',
       'F_Class', 'Bridge', 'Primary', 'Primary_link', 'Secondary',
       'Secondary_link', 'Tertiary', 'Tertiary_link', 'Trunk', 'Motorway',
       'Motorway_link', 'Service', 'Residential', 'Track', 'Unknown',
       'Unclassified', 'Final_Count', 'Final_Output', 'Predicted_Final',
       'Segment_Number'],
      dtype='object')
***************************
Total Segment = 81553
Length of Distance Array = 2176
Length of TimeNeeded_in_Seconds = 2176
Length of Charge_Consumed = 2176
Length of different Segments = 2176


Collecting Geometry (string with multiple coordinates) for each of the OSM Road Segment 

In [ ]:
from tqdm import tqdm
%cd /content/smarttransit-ai.github.io/files/dataset/OSM/
OSM_FILE ='chattanooga_osm.shp'
data = gpd.read_file(OSM_FILE)
fclass = list(data['fclass'])
OSM_FILE ='chattanooga_osm.shp'


CACHE_DIR = "./cache"



osm_gdf = gpd.GeoDataFrame.from_file(OSM_FILE)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR)
os.makedirs(CACHE_DIR)

spatial_index_file_name = "{}/{}.spatial".format(CACHE_DIR, "osm_network")
spatial_tree = index.Index(spatial_index_file_name)

OSM_BUFFER = np.add(0.00015,0.00005)
# OSM_BUFFER = 0.0002
GEO_Shapes_DICT = {}
osm_shps = []
with tqdm(total=len(list(osm_gdf.iterrows()))) as pbar:
    for i, gdf_row in osm_gdf.iterrows():
        shape = gdf_row['geometry']
        GEO_Shapes_DICT.update({gdf_row['osm_id']: gdf_row['geometry']})

        shape = gdf_row['geometry'].buffer(OSM_BUFFER) # Buffer it if needed.
        gdf_row['geometry'] = shape
        spatial_tree.insert(int(gdf_row["osm_id"]), shape.bounds, dict(gdf_row))
        # print(shape)
        osm_shps.append(shape)
        pbar.update(1)

bounds = spatial_tree.bounds

%cd /content/smarttransit-ai.github.io/files/dataset/Electric_Vehicles/
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')
    df = pd.read_csv(f'{Vehicle_Name}_with_Segment_Number_without_Turns.csv', low_memory=False)
    print(len(df))
    print(df.columns)

    OSM_ID = list(df.OSM_Feature)
    Geometry = []


    for i in range(len(OSM_ID)):
        OSM_ID[i] = str(OSM_ID[i])
        if i%10000 == 0:
            print(i)
        for key, value in GEO_Shapes_DICT.items():
            key = str(key)
            if key == OSM_ID[i] :
                Geometry.append(value)
                break
    print(len(Geometry))

    df['Geometry']=Geometry

    df.to_csv(f'{Vehicle_Name}_with_Segment_Number_without_Turns.csv',index=False)

/content/hdemma.github.io/Dataset/OSM


100%|██████████| 27807/27807 [00:28<00:00, 983.39it/s]


/content/hdemma.github.io/Dataset/Electric_Vehicles
Processing BYD_751
2176
Index(['Segment', 'Distance', 'OSM_Feature', 'Initial_recorded_Latitude',
       'Initial_recorded_Longitude', 'Final_recorded_Latitude',
       'Final_recorded_Longitude', 'TimeStamp_Begin', 'TimeStamp_End',
       'TimeStart', 'TimeFinish', 'Charge_at_Begining', 'Charge_at_Final',
       'TimeNeeded_in_Seconds', 'Primary', 'Primary_link', 'Secondary',
       'Secondary_link', 'Tertiary', 'Tertiary_link', 'Trunk', 'Motorway',
       'Motorway_link', 'Service', 'Residential', 'Track', 'Unknown',
       'Unclassified', 'Charge_Consumed'],
      dtype='object')
0
2176


Calculating Distance using each pair of coordinates from the geometry line string [Algorithm 2 from https://arxiv.org/abs/2004.06043]

In [ ]:
import tqdm

for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')
    df = pd.read_csv(f'{Vehicle_Name}_with_Segment_Number_without_Turns.csv', low_memory=False)
    print(len(df))
    print(df.columns)

    df = df.drop(df[df.Distance ==0].index & df[df.TimeNeeded_in_Seconds >10000].index & df[df.Charge_Consumed <-6].index)
    print(len(df))


    GEOM = list(df['Geometry'])
    Segment=list(df.Segment)


    lat_long_geometry = []



    for i in tqdm.tqdm(GEOM):
        lat_long_individual = []
        if len(i)>1:
            i = i.replace('LINESTRING ','')
            i = i.replace('(', '')
            i = i.replace(')', '')
            i = i.split(', ')
            for lat_long in i:
                lat_long = lat_long.split(' ')
                ll = []
                for x in lat_long:
                    x = float(x)
                    ll.append(x)
                lat_long_individual.append(ll)  #List of all the latitude-longitude for a sample
        lat_long_geometry.append(lat_long_individual)

    Distance_withLineSegment = []


    for j in tqdm.tqdm(range(len(GEOM))):
        if len(GEOM[j]) !=1:
            # line = shapely.wkt.loads(GEOM[j])           #Converting STRING into Shapely Object

            Initial_Point = Point(df.Initial_recorded_Latitude[j],df.Initial_recorded_Longitude[j])
            Final_Point = Point(df.Final_recorded_Latitude[j], df.Final_recorded_Longitude[j])


            LineSegment = []
            for x in range(len(lat_long_geometry[j]) - 1):

                x1 = Point(lat_long_geometry[j][x][1],lat_long_geometry[j][x][0])
                x2 = Point(lat_long_geometry[j][x+1][1],lat_long_geometry[j][x+1][0])
                LS = LineString([x1,x2])

                LineSegment.append(LS)

            distance_initial = []
            distance_final = []
            for ll in LineSegment:
                distance_initial.append(Initial_Point.distance(ll))
                distance_final.append(Final_Point.distance(ll))

            start_point = distance_initial.index(min(distance_initial))
            end_point = distance_final.index(min(distance_final))

            coords_1 = (df.Initial_recorded_Latitude[j], df.Initial_recorded_Longitude[j])
            coords_2 = (df.Final_recorded_Latitude[j], df.Final_recorded_Longitude[j])


            if start_point == end_point:

                D = geopy.distance.distance(coords_1, coords_2).m
                Distance_withLineSegment.append(D)


            elif start_point < end_point:
                min_dist_Line1_Start = LineSegment[start_point]
                
                coords = list(min_dist_Line1_Start.coords)

                roadsegment_individual = []
                for c in coords:
                    c = str(c).replace("(",'')
                    c = str(c).replace(")", '')
                    roadsegment_individual.append(c)

                fp2 = coords[1]

                d2 = geopy.distance.distance(coords_1, fp2).m

                D = d2

                for x in range(start_point,end_point,1):

                    if x > start_point:
                        Line1 = LineSegment[x]

                        coords = list(Line1.coords)
                        for c in coords:
                            c = str(c).replace("(", '')
                            c = str(c).replace(")", '')
                            roadsegment_individual.append(c)


                        fp1 = coords[0]
                        fp2 = coords[1]
                       
                        D = D + geopy.distance.distance(fp1, fp2).m
                        
                min_dist_Line2_End = LineSegment[end_point]
            
                coords = list(min_dist_Line2_End.coords)
                for c in coords:
                    c = str(c).replace("(", '')
                    c = str(c).replace(")", '')
                    roadsegment_individual.append(c)

                string='LINESTRING ('
                for substr in range(len(roadsegment_individual)):
                    roadsegment_individual[substr] = roadsegment_individual[substr].replace(',','')
                    if substr == 0:
                        string = string + roadsegment_individual[substr]
                    elif substr > 0 and substr < len(roadsegment_individual)-1:
                        string = string + ', ' + roadsegment_individual[substr]
                    else:
                        string = string + ', ' + roadsegment_individual[substr] + ')'

                fp1 = coords[0]
                fp2 = coords[1]

                d1 = geopy.distance.distance(coords_2, fp1).m

                D = D + d1
                Distance_withLineSegment.append(D)

            else:

                min_dist_Line1_Start = LineSegment[start_point]

                coords = list(min_dist_Line1_Start.coords)
                roadsegment_individual = []
                for c in coords:
                    c = str(c).replace("(", '')
                    c = c.replace(")", '')
                    roadsegment_individual.append(c)

                fp1 = coords[0]

                d1 = geopy.distance.distance(coords_1, fp1).m

                D = d1
                for x in range(start_point,end_point,-1):
                    if x < start_point:
                        Line1 = LineSegment[x]
                        coords = list(Line1.coords)
                        for c in coords:
                            c = str(c).replace("(", '')
                            c = str(c).replace(")", '')
                            roadsegment_individual.append(c)


                        fp1 = coords[0]
                        fp2 = coords[1]

                        D = D + geopy.distance.distance(fp1, fp2).m

                min_dist_Line2_End = LineSegment[end_point]
                coords = list(min_dist_Line2_End.coords)
                for c in coords:
                    c = str(c).replace("(", '')
                    c = str(c).replace(")", '')

                    roadsegment_individual.append(c)


                string = 'LINESTRING ('
                for substr in range(len(roadsegment_individual)):
                    roadsegment_individual[substr] = roadsegment_individual[substr].replace(',', '')
                    if substr == 0:
                        string = string + roadsegment_individual[substr]
                    elif substr > 0 and substr < len(roadsegment_individual) - 1:
                        string = string + ', ' + roadsegment_individual[substr]
                    else:
                        string = string + ', ' + roadsegment_individual[substr] + ')'

                fp2 = coords[1]

                d2 = geopy.distance.distance(coords_2, fp2).m
                D = D + d2
                Distance_withLineSegment.append(D)

        else:
            if df.Distance[j]==0:
                counter+=1
            Distance_withLineSegment.append(df.Distance[j])

        # print('******************************************************************\n')



    print(len(df.Distance))
    print(len(Distance_withLineSegment))

    df['Distance_withLineSegment'] = Distance_withLineSegment
    df.to_csv(f'{Vehicle_Name}_with_Segment_Number_with_New_Distance.csv',index=False)

Processing BYD_751
2176
Index(['Segment', 'Distance', 'OSM_Feature', 'Initial_recorded_Latitude',
       'Initial_recorded_Longitude', 'Final_recorded_Latitude',
       'Final_recorded_Longitude', 'TimeStamp_Begin', 'TimeStamp_End',
       'TimeStart', 'TimeFinish', 'Charge_at_Begining', 'Charge_at_Final',
       'TimeNeeded_in_Seconds', 'Primary', 'Primary_link', 'Secondary',
       'Secondary_link', 'Tertiary', 'Tertiary_link', 'Trunk', 'Motorway',
       'Motorway_link', 'Service', 'Residential', 'Track', 'Unknown',
       'Unclassified', 'Charge_Consumed', 'Geometry'],
      dtype='object')


  3%|▎         | 51/1605 [00:00<00:03, 501.94it/s]

1605
571
1605


100%|██████████| 1605/1605 [00:03<00:00, 526.92it/s]

1605
1605
1605
0


Preparing Samples for turns between 2 Road-Segments

In [ ]:
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')
    df = pd.read_csv(f'{Vehicle_Name}_with_Segment_Number_with_New_Distance.csv', low_memory=False)


    print(len(df))
    print(df.columns)


    NewSegment = []
    New_OSM_Feature = []

    New_Initial_Latitude = []
    New_Initial_Longitude = []
    New_Final_Latitude = []
    New_Final_Longitude = []

    Distance = []

    datetimeFormat = '%Y-%m-%d %H:%M:%S:%f'
    New_TimeStart = []
    New_TimeFinish = []
    New_TimeStamp_Begin = []
    New_TimeStamp_End = []

    TimeNeeded_in_Seconds = []

    New_Charge_at_Begining = []
    New_Charge_at_Final = []

    Charge_Consumed =[]


    #RoadTYpe



    primary=[]
    primary_link=[]
    secondary=[]
    secondary_link=[]
    tertiary=[]
    tertiary_link=[]

    trunk=[]
    trunk_link = []

    motorway=[]
    motorway_link=[]

    service=[]
    residential=[]

    track=[]
    track_grade2 = []

    path = []
    steps = []
    footway = []
    bridleway = []
    cycleway = []
    pedestrian = []
    living_street = []
    # unknown = []
    unclassified=[]


    Road_Type_TURN = []


    for i in range(len(df.Segment)):
        if i < len(df.Segment)-1:
            NewSegment.append(df.Segment[i])
            NewSegment.append(df.Segment[i]+1)
            New_OSM_Feature.append(df.OSM_Feature[i])
            New_OSM_Feature.append(df.OSM_Feature[i+1])

            New_Initial_Latitude.append(df.Initial_recorded_Latitude[i])
            New_Initial_Latitude.append(df.Final_recorded_Latitude[i])

            New_Initial_Longitude.append(df.Initial_recorded_Longitude[i])
            New_Initial_Longitude.append(df.Final_recorded_Longitude[i])

            New_Final_Latitude.append(df.Final_recorded_Latitude[i])
            New_Final_Latitude.append(df.Initial_recorded_Latitude[i + 1])

            New_Final_Longitude.append(df.Final_recorded_Longitude[i])
            New_Final_Longitude.append(df.Initial_recorded_Longitude[i+1])

            New_TimeStart.append(df.TimeStart[i])
            New_TimeStart.append(df.TimeFinish[i])

            New_TimeFinish.append(df.TimeFinish[i])
            New_TimeFinish.append(df.TimeStart[i+1])

            New_TimeStamp_Begin.append(df.TimeStamp_Begin[i])
            New_TimeStamp_Begin.append(df.TimeStamp_End[i])

            New_TimeStamp_End.append(df.TimeStamp_End[i])
            New_TimeStamp_End.append(df.TimeStamp_Begin[i + 1])

            New_Charge_at_Begining.append(df.Charge_at_Begining[i])
            New_Charge_at_Begining.append(df.Charge_at_Final[i])

            New_Charge_at_Final.append(df.Charge_at_Final[i])
            New_Charge_at_Final.append(df.Charge_at_Begining[i+1])


            #Calculate Time
            TimeNeeded_in_Seconds.append(df.TimeNeeded_in_Seconds[i])
            time_start = df.TimeFinish[i]
            time_end = df.TimeStart[i+1]


            TD = datetime.datetime.strptime(time_end, datetimeFormat) \
                 - datetime.datetime.strptime(time_start, datetimeFormat)

            TimeNeeded_in_Seconds.append(TD.seconds)

            #Calculate Charge

            Charge_Consumed.append(df.Charge_Consumed[i])
            initial_charge = df.Charge_at_Final[i]
            final_charge = df.Charge_at_Begining[i+1]

            C = initial_charge - final_charge
            Charge_Consumed.append(C)

            #Calculate Distance
            Distance.append(df.Distance_withLineSegment[i])


            coords_1 = (df.Final_recorded_Latitude[i],df.Final_recorded_Longitude[i])
            coords_2 = (df.Initial_recorded_Latitude[i+1],df.Initial_recorded_Longitude[i+1])
            D = geopy.distance.distance(coords_1, coords_2).m
            Distance.append(D)

            primary.append(df.primary[i])
            primary_link.append(df.primary_link[i])
            secondary.append(df.secondary[i])
            secondary_link.append(df.secondary_link[i])
            tertiary.append(df.tertiary[i])
            tertiary_link.append(df.tertiary_link[i])
            motorway.append(df.motorway[i])
            motorway_link.append(df.motorway_link[i])
            trunk.append(df.trunk[i])
            service.append(df.service[i])
            residential.append(df.residential[i])

            track.append(df.track[i])
            unclassified.append(df.unclassified[i])

            if df.primary[i] == 1 or df.primary[i + 1] == 1:
                primary.append(0.5)
            else:
                primary.append(0)

            if df.primary_link[i] == 1 or df.primary_link[i + 1] == 1:
                primary_link.append(0.5)
            else:
                primary_link.append(0)

            if df.secondary[i] == 1 or df.secondary[i + 1] == 1:
                secondary.append(0.5)
            else:
                secondary.append(0)

            if df.secondary_link[i] == 1 or df.secondary_link[i + 1] == 1:
                secondary_link.append(0.5)
            else:
                secondary_link.append(0)

            if df.tertiary[i] == 1 or df.tertiary[i + 1] == 1:
                tertiary.append(0.5)
            else:
                tertiary.append(0)

            if df.tertiary_link[i] == 1 or df.tertiary_link[i + 1] == 1:
                tertiary_link.append(0.5)
            else:
                tertiary_link.append(0)

            if df.trunk[i] == 1 or df.trunk[i + 1] == 1:
                trunk.append(0.5)
            else:
                trunk.append(0)

            if df.motorway[i] == 1 or df.motorway[i + 1] == 1:
                motorway.append(0.5)
            else:
                motorway.append(0)

            if df.motorway_link[i] == 1 or df.motorway_link[i + 1] == 1:
                motorway_link.append(0.5)
            else:
                motorway_link.append(0)

            if df.service[i] == 1 or df.service[i + 1] == 1:
                service.append(0.5)
            else:
                service.append(0)

            if df.residential[i] == 1 or df.residential[i + 1] == 1:
                residential.append(0.5)
            else:
                residential.append(0)

            if df.track[i] == 1 or df.track[i + 1] == 1:
                track.append(0.5)
            else:
                track.append(0)
            if df.unclassified[i] == 1 or df.unclassified[i + 1] == 1:
                unclassified.append(0.5)
            else:
                unclassified.append(0)

        else:
            NewSegment.append(df.Segment[i])
            New_OSM_Feature.append(df.OSM_Feature[i])

            New_Initial_Latitude.append(df.Initial_recorded_Latitude[i])

            New_Initial_Longitude.append(df.Initial_recorded_Longitude[i])

            New_Final_Latitude.append(df.Final_recorded_Latitude[i])

            New_Final_Longitude.append(df.Final_recorded_Longitude[i])

            New_TimeStart.append(df.TimeStart[i])

            New_TimeFinish.append(df.TimeFinish[i])

            New_TimeStamp_Begin.append(df.TimeStamp_Begin[i])
            New_TimeStamp_End.append(df.TimeStamp_End[i])

            New_Charge_at_Begining.append(df.Charge_at_Begining[i])

            New_Charge_at_Final.append(df.Charge_at_Final[i])

            # Calculate Time
            TimeNeeded_in_Seconds.append(df.TimeNeeded_in_Seconds[i])

            # Calculate Charge

            Charge_Consumed.append(df.Charge_Consumed[i])

            # Calculate Distance
            Distance.append(df.Distance_withLineSegment[i])

            primary.append(df.primary[i])
            primary_link.append(df.primary_link[i])
            secondary.append(df.secondary[i])
            secondary_link.append(df.secondary_link[i])
            tertiary.append(df.tertiary[i])
            tertiary_link.append(df.tertiary_link[i])
            motorway.append(df.motorway[i])
            motorway_link.append(df.motorway_link[i])

            trunk.append(df.trunk[i])

            service.append(df.service[i])
            residential.append(df.residential[i])

            track.append(df.track[i])
            # unknown.append(Old_unknown[i])
            unclassified.append(df.unclassified[i])



    NewSample=np.array(NewSegment)
    New_OSM_Feature = np.array(New_OSM_Feature)
    Distance = np.array(Distance)
    Initial_Latitude = np.array(New_Initial_Latitude)
    Initial_Longitude=np.array(New_Initial_Longitude)
    Final_Latitude=np.array(New_Final_Latitude)
    Final_Longitude=np.array(New_Final_Longitude)
    TimeStart=np.array(New_TimeStart)
    TimeFinish=np.array(New_TimeFinish)
    TimeStamp_Begin = np.array(New_TimeStamp_Begin)
    TimeStamp_End = np.array(New_TimeStamp_End)
    Charge_at_Begining=np.array(New_Charge_at_Begining)
    Charge_at_Final=np.array(New_Charge_at_Final)

    TimeNeeded_in_Seconds = np.array(TimeNeeded_in_Seconds)
    Charge_Consumed =  np.array(Charge_Consumed)
    # Type_of_Road = np.array(Type_of_Road)

    ######Road Types
    primary=np.array(primary)
    secondary_link=np.array(secondary_link)

    tertiary_link=np.array(tertiary_link)
    trunk=np.array(trunk)
    service=np.array(service)
    residential=np.array(residential)
    tertiary=np.array(tertiary)
    track=np.array(track)
    primary_link=np.array(primary_link)
    secondary=np.array(secondary)
    unclassified=np.array(unclassified)
    motorway_link=np.array(motorway_link)
    motorway=np.array(motorway)
    # Road_Type_TURN=np.array(Road_Type_TURN)


    TrainingSample = pd.DataFrame({"Sample": NewSample,
                                   "OSM_Feature":New_OSM_Feature,
                            "Distance":Distance,
                            "Initial_recorded_Latitude":Initial_Latitude,
                            "Initial_recorded_Longitude":Initial_Longitude,
                            "Final_recorded_Latitude":Final_Latitude,
                            "Final_recorded_Longitude":Final_Longitude,
                            "TimeStamp_Begin":TimeStamp_Begin,
                            "TimeStamp_End":TimeStamp_End,
                            "TimeStart":TimeStart,
                            "TimeFinish":TimeFinish,
                            "Energy_at_Begining":Charge_at_Begining,
                            "Energy_at_Final":Charge_at_Final,
                            "TimeNeeded_in_Seconds":TimeNeeded_in_Seconds,
                           "Primary": primary,
                           "Primary_link": primary_link,
                           "Secondary": secondary,
                           "Secondary_link": secondary_link,
                           "Tertiary": tertiary,
                           "Tertiary_link": tertiary_link,
                           "Trunk": trunk,
                           "Motorway": motorway,
                           "Motorway_link": motorway_link,
                           "Service": service,
                           "Residential": residential,
                           "Track": track,
                           "Unclassified": unclassified,
                            "Energy_Consumed":Charge_Consumed})
    
    print(len(TrainingSample))

    TrainingSample.to_csv(f'{Vehicle_Name}_with_all_Samples.csv',index=False)

Processing BYD_751
1605
Index(['Segment', 'Distance', 'OSM_Feature', 'Initial_recorded_Latitude',
       'Initial_recorded_Longitude', 'Final_recorded_Latitude',
       'Final_recorded_Longitude', 'TimeStamp_Begin', 'TimeStamp_End',
       'TimeStart', 'TimeFinish', 'Charge_at_Begining', 'Charge_at_Final',
       'TimeNeeded_in_Seconds', 'Primary', 'Primary_link', 'Secondary',
       'Secondary_link', 'Tertiary', 'Tertiary_link', 'Trunk', 'Motorway',
       'Motorway_link', 'Service', 'Residential', 'Track', 'Unknown',
       'Unclassified', 'Charge_Consumed', 'Geometry',
       'Distance_withLineSegment'],
      dtype='object')
3209


Removing Outliers on Conditions
1. Time Interval < 1 hour
2. Distance Travelled == 0
3. Energy Consumed ==0 or Energy Consumed < -0.2 or Energy Consumed >10
4. Distance Travelled is way greater/less than Time Taken 

In [ ]:
for v in Vehicle_ID:
    Vehicle_Name = f'BYD_{v}'
    print(f'Processing {Vehicle_Name}')
    df = pd.read_csv(f'{Vehicle_Name}_with_all_Samples.csv', low_memory=False)

    print(len(df))
    
    TimeNeeded_in_Seconds = list(df.TimeNeeded_in_Seconds)
    data = df.drop(df[df.TimeNeeded_in_Seconds > 3600].index)
    print(len(data))


    df = data.drop(data[data.Energy_Consumed ==0].index | data[data.Energy_Consumed < -0.2].index | data[data.Energy_Consumed > 10].index)
    print(len(df))

    df2 = df.drop(df[data.Distance == 0].index)
    print(len(df2))

    df2 = df2.drop(df2[df2.Distance > 2000].index & df2[df2.TimeNeeded_in_Seconds < 60].index)
    print(len(df2))
    data = df2.drop(df2[df2.Distance < 100].index & df2[df2.TimeNeeded_in_Seconds > 2000].index)
    print(len(data))

    df.to_csv(f'{Vehicle_Name}_with_all_Samples_without_Outliers.csv',index=False)

Processing BYD_751
3209
3207
3205
3015
3015
3015


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
